In [1]:
!pip install groq python-dotenv numpy tqdm datasets

In [5]:
from groq import Groq
from dotenv import load_dotenv
from datasets import load_dataset

import os
from tqdm import tqdm
import re
import random
import pprint

from typing import List, Dict, Any

load_dotenv()
random.seed(0)

client = Groq()
gsm8k_dataset = load_dataset("gsm8k", "main")

gsm8k_train = gsm8k_dataset["train"]
gsm8k_test  = gsm8k_dataset["test"]

In [6]:
def generate_response_using_Llama(
        prompt: str,
        model: str = "llama-3.1-8b-instant"
    ):
    try:
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    "content": "You are a helpful assistant that solves math problems."
                },
                {
                    "role": "user", 
                    "content": prompt
                }
            ],
            model=model,
            temperature=0.3, ### 수정해도 됩니다!
            stream=False
        )
        return chat_completion.choices[0].message.content
    
    except Exception as e:
        print(f"API call error: {str(e)}")
        return None

#### 응답 잘 나오는지 확인해보기

In [7]:
response = generate_response_using_Llama(
    prompt="Hello world!",
)
print(response)

Hello world. I'm here to help with any math problems you'd like to solve. What kind of math are you working on today?


#### GSM8K 데이터셋 확인해보기

In [8]:
print("[Question]")
for l in gsm8k_test['question'][0].split("."):
    print(l)
print("="*100)
print("[Answer]")
print(gsm8k_test['answer'][0])

[Question]
Janet’s ducks lay 16 eggs per day
 She eats three for breakfast every morning and bakes muffins for her friends every day with four
 She sells the remainder at the farmers' market daily for $2 per fresh duck egg
 How much in dollars does she make every day at the farmers' market?
[Answer]
Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.
She makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.
#### 18


#### Util 함수들
- extract_final_answer: LLM의 응답을 parse하여 최종 결과만 추출 (정답과 비교하기 위해)
- run_benchmark_test: 벤치마크 테스트
- save_final_result: 결과물 제출을 위한 함수

In [10]:
### 수정해도 됩니다!
def extract_final_answer(response: str):
    regex = r"(?:Answer:|Model response:)\s*\$?([0-9,]+)\b|([0-9,]+)\s*(meters|cups|miles|minutes)"
    matches = re.finditer(regex, response, re.MULTILINE)
    results = [match.group(1) if match.group(1) else match.group(2).replace(",", "") for match in matches]

    if len(results) == 0:
        additional_regex = r"\$?([0-9,]+)"
        additional_matches = re.finditer(additional_regex, response, re.MULTILINE)
        results.extend([match.group(1).replace(",", "") for match in additional_matches])

    return results[-1] if results else None

In [9]:
### 수정해도 됩니다!
import time

def run_benchmark_test(
        dataset,
        prompt: str,
        model: str = "llama3-8b-8192",
        num_samples: int = 50,
        VERBOSE: bool = False
    ):
    correct = 0
    total   = 0
    results = []

    for i in tqdm(range(min(num_samples, len(dataset)))):
        time.sleep(2) # RPM 30 제한 피하기 위한 것

        question = dataset[i]["question"]
        correct_answer = float(re.findall(r'\d+(?:\.\d+)?', dataset[i]["answer"].split('####')[-1])[0])

        response = generate_response_using_Llama(
            prompt=prompt.format(question=question),
            model=model
        )

        if response:
            predicted_answer_str = extract_final_answer(response)
            try:
                predicted_answer = float(str(predicted_answer_str).replace(",", "")) if predicted_answer_str else None
            except ValueError:
                predicted_answer = None

            is_correct = False
            if predicted_answer is not None:
                is_correct = abs(predicted_answer - correct_answer) < 1e-5
            
            if is_correct:
                correct += 1
            total += 1
            
            results.append({
                'question': question,
                'correct_answer': correct_answer,
                'predicted_answer': predicted_answer,
                'response': response,
                'correct': is_correct
            })

            if (i + 1) % 5 == 0:
                current_acc = correct/total if total > 0 else 0
                print(f"Progress: [{i+1}/{num_samples}]")
                print(f"Current Acc.: [{current_acc:.2%}]")

    return results, correct/total if total > 0 else 0

In [11]:
def save_final_result(results: List[Dict[str, Any]], accuracy: float, filename: str) -> None:
    result_str = f"====== ACCURACY: {accuracy} ======\n\n"
    result_str += f"[Details]\n"
    
    for idx, result in enumerate(results):
        result_str += f"Question {idx+1}: {result['question']}\n"
        result_str += f"Correct Answer: {result['correct_answer']}\n"
        result_str += f"Predicted Answer: {result['predicted_answer']}\n"
        result_str += f"Correct: {result['correct']}\n\n"
    
    with open(filename, "w", encoding="utf-8") as f:
        f.write(result_str)

#### Direct prompting with few-shot example

In [ ]:
def construct_direct_prompt(num_examples: int = 3) -> str:
    train_dataset = gsm8k_train

    sampled_indices = random.sample(
        [i for i in range(len(train_dataset['question']))],
        num_examples
    )

    prompt = "Instruction:\nSolve the following mathematical question and generate ONLY the answer after a tag, 'Answer:' without any rationale.\n"

    for i in range(num_examples):
        cur_question = train_dataset['question'][i]
        cur_answer = train_dataset['answer'][i].split("####")[-1].strip()

        prompt += f"\n[Example {i+1}]\n"
        prompt += f"Question:\n{cur_question}\n"
        prompt += f"Answer:{cur_answer}\n"

    prompt += "\nQuestion:\n{question}\nAnswer:"

    return prompt

In [ ]:
### 어떤 방식으로 저장되는지 확인해보세요!
PROMPT = construct_direct_prompt(3)
VERBOSE = False

results, accuracy = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=PROMPT,
    # model="llama-3.1-8b-instant",
    VERBOSE=VERBOSE,
    num_samples=10
)
save_final_result(results, accuracy, "example.txt")

 10%|█         | 1/10 [00:02<00:19,  2.21s/it]

API call error: Error code: 400 - {'error': {'message': 'The model `llama3-8b-8192` has been decommissioned and is no longer supported. Please refer to https://console.groq.com/docs/deprecations for a recommendation on which model to use instead.', 'type': 'invalid_request_error', 'code': 'model_decommissioned'}}


 20%|██        | 2/10 [00:04<00:17,  2.20s/it]

API call error: Error code: 400 - {'error': {'message': 'The model `llama3-8b-8192` has been decommissioned and is no longer supported. Please refer to https://console.groq.com/docs/deprecations for a recommendation on which model to use instead.', 'type': 'invalid_request_error', 'code': 'model_decommissioned'}}


 30%|███       | 3/10 [00:06<00:15,  2.24s/it]

...

API call error: Error code: 400 - {'error': {'message': 'The model `llama3-8b-8192` has been decommissioned and is no longer supported. Please refer to https://console.groq.com/docs/deprecations for a recommendation on which model to use instead.', 'type': 'invalid_request_error', 'code': 'model_decommissioned'}}


 90%|█████████ | 9/10 [00:20<00:02,  2.39s/it]

API call error: Error code: 400 - {'error': {'message': 'The model `llama3-8b-8192` has been decommissioned and is no longer supported. Please refer to https://console.groq.com/docs/deprecations for a recommendation on which model to use instead.', 'type': 'invalid_request_error', 'code': 'model_decommissioned'}}


100%|██████████| 10/10 [00:23<00:00,  2.32s/it]

API call error: Error code: 400 - {'error': {'message': 'The model `llama3-8b-8192` has been decommissioned and is no longer supported. Please refer to https://console.groq.com/docs/deprecations for a recommendation on which model to use instead.', 'type': 'invalid_request_error', 'code': 'model_decommissioned'}}


In [ ]:
# TODO: 0 shot, 3 shot, 5 shot direct prompting을 통해 벤치마크 테스트를 한 후, 각각 direct_prompting_{shot: int}.txt로 저장해주세요!
# 예시: shot이 5인 경우 direct_prompting_5.txt
# 항상 num_samples=50 입니다!

shots = [0, 3, 5]

for shot in shots:
    print(f"****Running Direct Prompting: {shot} Shot****")

    current_prompt = construct_direct_prompt(shot)
    
    # 테스트 실행 (num_samples=50) 
    res, acc = run_benchmark_test(
        dataset=gsm8k_test,
        prompt=current_prompt,
        model="llama-3.1-8b-instant",
        num_samples=50
    )
    
    # 결과 저장
    file_name = f"direct_prompting_{shot}.txt"
    save_final_result(res, acc, file_name)
    print(f"Saved: {file_name} (Accuracy: {acc:.2%})")

****Running Direct Prompting: 0 Shot****


 10%|█         | 5/50 [00:13<01:57,  2.62s/it]

Progress: [5/50]
Current Acc.: [80.00%]


 20%|██        | 10/50 [00:26<01:41,  2.55s/it]

Progress: [10/50]
Current Acc.: [70.00%]


...

Progress: [40/50]
Current Acc.: [80.00%]


 90%|█████████ | 45/50 [04:02<00:26,  5.30s/it]

Progress: [45/50]
Current Acc.: [75.56%]


100%|██████████| 50/50 [04:28<00:00,  5.38s/it]

Progress: [50/50]
Current Acc.: [76.00%]
Saved: direct_prompting_5.txt (Accuracy: 76.00%)


### Chain-of-Thought prompting with few-shot example
```text
[Question]
Janet’s ducks lay 16 eggs per day
 She eats three for breakfast every morning and bakes muffins for her friends every day with four
 She sells the remainder at the farmers' market daily for $2 per fresh duck egg
 How much in dollars does she make every day at the farmers' market?
====================================================================================================
[Answer]
Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.
She makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.
#### 18
```

[Answer] 아래의 정답을 도출하는 과정을 예시로 달아주면 CoT의 few shot이 되겠죠?

In [ ]:
def construct_CoT_prompt(num_examples: int = 3) -> str:
    train_dataset = gsm8k_train

    sampled_indices = random.sample(
        [i for i in range(len(train_dataset['question']))],
        num_examples
    )
    prompt = (
        "Instruction:\n"
        "Solve the following mathematical questions step by step. "
        "Provide the full reasoning and then give the final answer after the tag '#### '.\n"
    )

    for i, idx in enumerate(sampled_indices):
        cur_question = train_dataset[idx]['question']
        cur_answer = train_dataset[idx]['answer']

        prompt += f"\n[Example {i+1}]\n"
        prompt += f"Question:\n{cur_question}\n"
        prompt += f"Answer:\n{cur_answer}\n"

    prompt += "\nQuestion:\n{question}\nAnswer:"
    return prompt

In [ ]:
# TODO: 0 shot, 3 shot, 5 shot CoT prompting을 통해 벤치마크 테스트를 한 후, 각각 CoT_prompting_{shot: int}.txt로 저장해주세요!
# 예시: shot이 5인 경우 CoT_prompting_5.txt
# 항상 num_samples=50 입니다!

for shot in [0, 3, 5]:
    print(f"**** Running CoT Prompting: {shot} Shot ****")
    prompt_text = construct_CoT_prompt(shot)
    
    results, accuracy = run_benchmark_test(
        dataset=gsm8k_test,
        prompt=prompt_text,
        model="llama-3.1-8b-instant",
        num_samples=50
    )

    save_final_result(results, accuracy, f"CoT_prompting_{shot}.txt")
    print(f"Saved: CoT_prompting_{shot}.txt (Accuracy: {accuracy:.2%})")

**** Running CoT Prompting: 0 Shot ****


 10%|█         | 5/50 [00:14<02:04,  2.77s/it]

Progress: [5/50]
Current Acc.: [80.00%]


 20%|██        | 10/50 [00:27<01:46,  2.67s/it]

Progress: [10/50]
Current Acc.: [60.00%]


...

Progress: [40/50]
Current Acc.: [62.50%]


 90%|█████████ | 45/50 [01:56<00:13,  2.63s/it]

Progress: [45/50]
Current Acc.: [62.22%]


100%|██████████| 50/50 [02:09<00:00,  2.60s/it]

Progress: [50/50]
Current Acc.: [64.00%]
Saved: CoT_prompting_5.txt (Accuracy: 64.00%)


### Construct your prompt!!

목표: 본인만의 프롬프트를 통해 정답률을 더 끌어올려보기!
- gsm8k의 train 데이터셋에서 예시를 가져온 다음 (자유롭게!)
- 그 예시들에 대한 풀이 과정을 만들어주세요!
- 모든 것들이 자유입니다! Direct Prompting, CoT Prompting을 한 결과보다 정답률만 높으면 돼요.

In [ ]:
def construct_my_prompt(num_examples: int = 3) -> str:
    train_dataset = gsm8k_train
    # 실험적으로 성능이 좋았던 고정 예시 인덱스나 랜덤 샘플링 사용
    sampled_indices = random.sample(range(len(train_dataset)), num_examples)

    # 지시사항을 간결하고 강력하게 변경
    prompt = (
        "Instruction:\n"
        "Solve the following grade school math problem. \n"
        "Examine the question carefully, write your step-by-step solution, and provide the final answer.\n"
        "Your final answer must be a single number preceded by 'Answer: '.\n"
    )

    for i, idx in enumerate(sampled_indices):
        cur_question = train_dataset[idx]['question']
        raw_answer = train_dataset[idx]['answer']
        
        # CoT 풀이에서 <<...>> 태그를 제거하여 가독성 증대
        reasoning = re.sub(r"<<.*?>>", "", raw_answer.split('####')[0].strip())
        final_val = raw_answer.split('####')[-1].strip()

        prompt += f"\nExample {i+1}:\n"
        prompt += f"Question: {cur_question}\n"
        prompt += f"Solution: {reasoning}\n"
        prompt += f"Answer: {final_val}\n"

    prompt += "\nQuestion: {question}\nSolution:"
    return prompt

In [20]:
# TODO: 만든 0 shot, 3 shot, 5 shot example과 프롬프트를 통해 벤치마크 테스트를 한 후, 각각 My_prompting_{shot: int}.txt로 저장해주세요!
# 예시: shot이 5인 경우 My_prompting_5.txt
# 항상 num_samples=50 입니다!

for shot in [0, 3, 5]:
    print(f"**** Running My Prompting (Improved): {shot} Shot ****")
    prompt_text = construct_my_prompt(shot)
    results, accuracy = run_benchmark_test(
        dataset=gsm8k_test,
        prompt=prompt_text,\
        model="llama-3.1-8b-instant",
        num_samples=50
    )
    save_final_result(results, accuracy, f"My_prompting_{shot}.txt")
    print(f"Final Acc: {accuracy:.2%}")

**** Running My Prompting (Improved): 0 Shot ****


 10%|█         | 5/50 [01:31<18:22, 24.50s/it]

Progress: [5/50]
Current Acc.: [100.00%]


 20%|██        | 10/50 [03:56<19:14, 28.86s/it]

Progress: [10/50]
Current Acc.: [80.00%]


...

API call error: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.1-8b-instant` in organization `org_01kgbdfh72fb1sbxh5ek4fevpd` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 499561, Requested 837. Please try again in 1m8.7744s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}


 98%|█████████▊| 49/50 [03:53<00:15, 15.12s/it]

API call error: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.1-8b-instant` in organization `org_01kgbdfh72fb1sbxh5ek4fevpd` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 499985, Requested 784. Please try again in 2m12.8832s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}


100%|██████████| 50/50 [03:55<00:00,  4.71s/it]

API call error: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.1-8b-instant` in organization `org_01kgbdfh72fb1sbxh5ek4fevpd` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 499973, Requested 790. Please try again in 2m11.846399999s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}
Final Acc: 100.00%


In [21]:
import pandas as pd

# 결과 데이터를 담을 리스트
summary_data = []

techniques = ["direct_prompting", "CoT_prompting", "My_prompting"]
shots = [0, 3, 5]

for tech in techniques:
    row = {"Technique": tech}
    for shot in shots:
        filename = f"{tech}_{shot}.txt"
        if os.path.exists(filename):
            with open(filename, "r", encoding="utf-8") as f:
                content = f.readline() # 첫 번째 줄에 ACCURACY 정보가 있음
                # "====== ACCURACY: 0.82 ======" 에서 숫자만 추출
                acc_match = re.search(r"ACCURACY: ([\d.]+)", content)
                accuracy = float(acc_match.group(1)) if acc_match else 0.0
                row[f"{shot}-Shot Acc"] = f"{accuracy:.2%}"
        else:
            row[f"{shot}-Shot Acc"] = "N/A"
    summary_data.append(row)

# 데이터프레임 생성 및 출력
df_summary = pd.DataFrame(summary_data)
print("\n[ Final Accuracy Summary Table ]")
display(df_summary) # 주피터 노트북에서 예쁜 표로 출력


[ Final Accuracy Summary Table ]


,Technique,0-Shot Acc,3-Shot Acc,5-Shot Acc
0,direct_prompting,82.00%,82.00%,76.00%
1,CoT_prompting,64.00%,70.00%,64.00%
2,My_prompting,84.00%,100.00%,100.00%


### 보고서 작성하기
#### 아래의 내용이 포함되면 됩니다!

1. Direct Prompting, CoT Prompting, My Prompting을 0 shot, 3 shot, 5 shot 정답률을 표로 보여주세요!
2. CoT Prompting이 Direct Prompting에 비해 왜 좋을 수 있는지에 대해서 서술해주세요!
3. 본인이 작성한 프롬프트 기법이 CoT에 비해서 왜 더 좋을 수 있는지에 대해서 설명해주세요!
4. 최종적으로, `PROMPTING.md`에 보고서를 작성해주세요!